
# Fitness Tracker App (Notebook Version)

This notebook lets you:
- Track workouts and append them to a CSV log
- View and plot your progress
- **Start fresh** by deleting logs with an optional backup

> Tip: Run the cells in order the first time.


In [10]:
import os
import sys
import math
import json
import time
import shutil
import zipfile
import datetime as dt
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, clear_output
import ipywidgets as W
import matplotlib.dates as mdates

try:
    from send2trash import send2trash
    HAS_SEND2TRASH = True
except Exception:
    HAS_SEND2TRASH = False

LOG_DIR = Path("logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)

LOG_FILE = LOG_DIR / "workouts.csv"

def append_to_log(df: pd.DataFrame, log_file: Path = LOG_FILE):
    log_file.parent.mkdir(parents=True, exist_ok=True)
    header = not log_file.exists()
    df.to_csv(log_file, mode="a", header=header, index=False)

def read_log(log_file: Path = LOG_FILE) -> pd.DataFrame:
    if log_file.exists():
        try:
            return pd.read_csv(log_file)
        except Exception as e:
            print(f"Could not read log: {e}")

    return pd.DataFrame(columns=["date", "exercise", "weight_kg", "reps", "duration_sec", "notes"])


In [3]:

# ==== Delete logs ====

def _zip_backup(src_dir: Path) -> Optional[Path]:
    """Create a timestamped zip backup of the log directory. Returns path or None if dir empty."""
    files = [p for p in src_dir.rglob("*") if p.is_file()]
    if not files:
        return None
    ts = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
    backup_path = src_dir.parent / f"{src_dir.name}-backup-{ts}.zip"
    with zipfile.ZipFile(backup_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for f in files:
            zf.write(f, arcname=f.relative_to(src_dir))
    return backup_path

def delete_logs(
    log_dir: Path = LOG_DIR,
    patterns: Tuple[str, ...] = ("*.csv", "*.json", "*.parquet", "*.db", "*.sqlite"),
    backup: bool = True,
    soft_delete: bool = True
) -> Tuple[int, Optional[Path]]:
    """Deletes files in LOG_DIR matching patterns. Returns (#deleted, backup_zip_path)."""
    backup_path = _zip_backup(log_dir) if backup else None

    count = 0
    for pat in patterns:
        for f in log_dir.rglob(pat):
            try:
                if soft_delete and HAS_SEND2TRASH:
                    send2trash(str(f))
                else:
                    f.unlink(missing_ok=True)
                count += 1
            except Exception as e:
                print(f"Could not delete {f}: {e}")

    for d in sorted({p.parent for p in log_dir.rglob("*")}, reverse=True):
        if d.exists():
            try:
                d.rmdir()
            except OSError:
                pass  # not empty

    return count, backup_path

confirm = W.Checkbox(description="I understand this will delete my logs.", value=False)
backup = W.Checkbox(description="Create a backup ZIP before deleting", value=True)
soft   = W.Checkbox(description="Send to Recycle Bin (if available)", value=True)
btn    = W.Button(description="Delete logs", button_style="danger")
out    = W.Output()

def _on_click(_):
    with out:
        clear_output()
        if not confirm.value:
            print("Please tick the confirmation box first.")
            return
        n, zip_path = delete_logs(backup=backup.value, soft_delete=soft.value)
        msg = f"Deleted {n} file(s)."
        if zip_path is not None:
            msg += f" Backup saved to: {zip_path}"
        print(msg)

btn.on_click(_on_click)

display(W.VBox([
    W.HTML("<b>Start fresh?</b> Use this tool to clear your logs."),
    W.HBox([confirm]),
    W.HBox([backup, soft]),
    btn,
    out
]))



## Add a Workout
Fill the form and click **Save Entry** to append to your log.


In [13]:

# ==== Add workout entry form ====

date_picker   = W.DatePicker(description="Date")
exercise_dd   = W.Text(description="Exercise", placeholder="e.g., Squat")
weight_in     = W.BoundedFloatText(description="Weight (kg)", min=0, max=1000, step=0.5, value=0.0)
reps_in       = W.BoundedIntText(description="Reps", min=0, max=1000, value=0)
duration_in   = W.BoundedIntText(description="Duration (sec)", min=0, max=36000, value=0)
notes_in      = W.Textarea(description="Notes", placeholder="optional")
save_btn      = W.Button(description="Save Entry", button_style="success")
save_out      = W.Output()

def _save_entry(_):
    with save_out:
        clear_output()
        d = date_picker.value or dt.date.today()
        row = {
            "date": d.isoformat(),
            "exercise": exercise_dd.value.strip() or "Unknown",
            "weight_kg": float(weight_in.value),
            "reps": int(reps_in.value),
            "duration_sec": int(duration_in.value),
            "notes": notes_in.value.strip(),
        }
        df = pd.DataFrame([row])
        append_to_log(df)
        print("Entry saved:", row)

save_btn.on_click(_save_entry)

display(W.VBox([
    date_picker,
    exercise_dd,
    weight_in,
    reps_in,
    duration_in,
    notes_in,
    save_btn,
    save_out
]))



## View Log
Run the cell below to load and display the current log.


In [14]:

df_log = read_log()
display(df_log.head(50))
print(f"Rows: {len(df_log)}")


,date,exercise,weight_kg,reps,duration_sec,notes
0,2025-11-01,squat,100.0,10,20,NaN
1,2025-11-01,leg press,100.0,10,20,NaN
2,2025-11-02,leg press,90.0,12,20,NaN
3,2025-11-02,squat,140.0,8,20,NaN
4,2025-11-03,squat,140.0,10,20,NaN
5,2025-11-03,leg press,200.0,10,20,NaN


Rows: 6



## Plot Progress
Select an exercise to plot **weight (kg)** over time.


In [15]:

df_log = read_log()
if df_log.empty:
    print("No data to plot yet. Add some entries first.")
else:
    exs = sorted([e for e in df_log["exercise"].dropna().unique() if str(e).strip()])
    ex = W.Dropdown(options=exs, description="Exercise")
    btnp = W.Button(description="Plot")
    outp = W.Output()

    def _do_plot(_):
        with outp:
            clear_output()
            sub = df_log[df_log["exercise"] == ex.value].copy()
            if sub.empty:
                print("No rows for this exercise.")
                return
            sub["date"] = pd.to_datetime(sub["date"], errors="coerce")
            sub = sub.dropna(subset=["date"])
            sub = sub.sort_values("date")
            plt.figure()
            plt.plot(sub["date"], sub["weight_kg"], marker="o")
            plt.title(f"Weight over time — {ex.value}")
            plt.xlabel("Date")
            plt.ylabel("Weight (kg)")
            plt.grid(True)
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
            plt.gcf().autofmt_xdate()
            plt.show()

    btnp.on_click(_do_plot)
    display(W.HBox([ex, btnp]), outp)


Output()